In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os 
from sklearn.preprocessing import RobustScaler,StandardScaler, LabelEncoder, OneHotEncoder
import cv2
import mediapipe as mp
import pickle

In [2]:
data_dir= './archive/asl_alphabet_train/asl_alphabet_train'
data=[]
labels=[]

In [52]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.01)

In [5]:
for dir in os.listdir(data_dir):
    print(dir)
    for img_path in os.listdir(os.path.join(data_dir,dir)):
        data_aux=[]
        x_list=[]
        y_list=[]
        img = cv2.imread(os.path.join(data_dir, dir, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_list.append(x)
                    y_list.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_list))
                    data_aux.append(y - min(y_list))
            while len(data_aux)<42:
                data_aux.append(0)
            data.append(data_aux)
            labels.append(dir)
f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

A
B
C
D
del
E
F
G
H
I
J
K
L
M
N
nothing
O
P
Q
R
S
space
T
U
V
W
X
Y
Z


In [3]:
data_dict = pickle.load(open('data.pickle', 'rb'))
len(data_dict['labels'])
data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

In [4]:
np.unique(data_dict['labels'])

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'del', 'nothing', 'space'], dtype='<U7')

In [5]:
from collections import Counter

# Count occurrences of each distinct value
occurrences = Counter(data_dict['labels'])

print(occurrences)

Counter({'A': 4423, 'F': 2891, 'K': 2743, 'J': 2656, 'S': 2652, 'Y': 2638, 'R': 2609, 'V': 2605, 'L': 2589, 'D': 2575, 'G': 2574, 'U': 2571, 'W': 2524, 'H': 2477, 'I': 2470, 'Z': 2440, 'T': 2389, 'O': 2361, 'E': 2353, 'B': 2257, 'X': 2239, 'Q': 2192, 'P': 2108, 'C': 2031, 'M': 1869, 'space': 1849, 'del': 1819, 'N': 1506, 'nothing': 2})


In [6]:
num_classes =29

In [7]:
model = keras.models.Sequential([
    tf.keras.layers.Input(shape=(42,)),  # 21 landmarks with x, y, z coordinates for a single hand
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
encoder=LabelEncoder()
labels=encoder.fit_transform(labels)
from tensorflow.keras.utils import to_categorical

labels = to_categorical(labels, num_classes=29)

In [9]:
labels.shape

(68412, 29)

In [10]:
model.fit(x=data,
          y=labels,
         epochs= 10)

Epoch 1/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7106 - loss: 1.1565
Epoch 2/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9600 - loss: 0.1898
Epoch 3/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9721 - loss: 0.1257
Epoch 4/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9759 - loss: 0.1019
Epoch 5/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9780 - loss: 0.0911
Epoch 6/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9822 - loss: 0.0762
Epoch 7/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9832 - loss: 0.0695
Epoch 8/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9842 - loss: 0.0623
Epoch 9/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9843 - loss: 0.0601
Epoch 10/10
2138/2138 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9858 - loss: 0.0561


In [11]:
test_dict = pickle.load(open('test.pickle', 'rb'))

test_data = np.asarray(test_dict['data'])
test_labels = np.asarray(test_dict['labels'])

In [12]:
labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
        'del', 'nothing', 'space']

In [15]:
predictions = model.predict(test_data)
predicted_class = np.argmax(predictions, axis=1)
for pred in predicted_class:
    print(labels[pred])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
A
B
C
D
F
G
H
I
K
L
M
O
P
Q
R
space
S
S
W
S
Y


In [ ]:
f = open('model6.p', 'wb')
pickle.dump({'model': model}, f)
f.close()